# Aplicaciones Prácticas de NLP con Transformers

## Guía de Ejercicios Progresivos

---

## Introducción

En este cuaderno vas a trabajar con aplicaciones reales de Procesamiento de Lenguaje Natural (NLP) utilizando modelos Transformer preentrenados. Cada ejercicio representa un caso de uso concreto que se implementa en empresas y organizaciones de Argentina y el mundo.

Los ejercicios están organizados de manera progresiva:

1. **Nivel Básico:** Moderación automática de comentarios en redes sociales
2. **Nivel Intermedio:** Extracción de información de currículums (RRHH)
3. **Nivel Avanzado:** Chatbot de soporte técnico con respuestas automáticas
4. **Desafío Autónomo:** Análisis integral de reseñas de restaurantes

### ¿Qué vas a aprender?

- A usar modelos preentrenados de Hugging Face para tareas reales
- A comprender cómo funcionan los pipelines de NLP en la práctica
- A modificar y experimentar con código para resolver problemas concretos
- A evaluar la calidad de las predicciones de los modelos
- A trabajar de manera autónoma en un problema real completo

### Requisitos previos

- Conocimientos básicos de Python
- Familiaridad con conceptos de Machine Learning
- Haber visto los cuadernos introductorios sobre clasificación de sentimientos

---

## Instalación de dependencias

Ejecutá la siguiente celda para instalar las librerías necesarias:

In [ ]:
# Instalación de librerías (ejecutar solo una vez)
!pip install transformers torch pandas -q
!pip uninstall transformers tokenizers accelerate -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers accelerate tokenizers

Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 994.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.0 MB/s eta 0:00:00
     ━━━━━━

---

## Ejercicio 1: Moderación de Comentarios en Redes Sociales

### Contexto del problema

Imaginate que trabajás en el equipo de redes sociales de una empresa argentina de delivery de comida. Recibís cientos de comentarios diarios en Instagram y Facebook. Necesitás un sistema que identifique automáticamente los comentarios negativos o críticos para que el equipo de atención al cliente pueda responderlos con prioridad.

### Aplicación real

Este tipo de sistemas se usan en:
- Redes sociales de empresas (PedidosYa, Rappi, Mercado Libre)
- Plataformas de e-commerce
- Servicios de atención al cliente automatizada

### ¿Qué vamos a hacer?

1. Cargar un modelo de análisis de sentimientos en español
2. Analizar comentarios reales de clientes
3. Clasificarlos automáticamente (positivo, negativo, neutral)
4. Identificar cuáles requieren atención urgente

---

### Paso 1: Importar librerías y cargar el modelo

In [ ]:
from transformers import pipeline
import pandas as pd

# Cargamos un modelo de análisis de sentimientos específico para español
# Este modelo fue entrenado con datos de redes sociales en español
clasificador = pipeline(
    "text-classification",
    model="finiteautomata/beto-sentiment-analysis"
)

print("Modelo cargado correctamente.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Modelo cargado correctamente.


In [ ]:
!pip install --upgrade transformers torch -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.7.1+cu118 requires torch==2.7.1, but you have torch 2.9.0 which is incompatible.
torchvision 0.22.1+cu118 requires torch==2.7.1, but you have torch 2.9.0 which is incompatible.
fastai 2.8.4 requires torch<2.9,>=1.10, but you have torch 2.9.0 which is incompatible.


After running the upgrade, please run the cell again that caused the error (cell `3RtrDupFp1Q5`).

### Paso 2: Definir comentarios de ejemplo

Acá tenemos comentarios típicos que podría recibir una empresa de delivery:

In [ ]:
comentarios = [
    "La comida llegó rapidísimo y estaba caliente. El repartidor re amable, excelente servicio!",
    "Tardaron DOS HORAS para traer una pizza fría. Pésimo, no pido más acá.",
    "Todo bien, llegó en tiempo y forma. Nada para destacar.",
    "Me encanta este servicio, siempre cumplen. Los re banco!",
    "Se equivocaron con mi pedido OTRA VEZ. Ya es la tercera vez que pasa, un desastre total.",
    "Increíble, dos horas esperando que lo entreguen y la pizza llega con el queso pegado en la caja.",
    "Llegó todo correctamente.",
    "Me llegó una empanada mordida, sin comentarios....",
    "La empanada era pequeña.",
    "Me dijeron que no pida acá pero me encantó"
]

### Paso 3: Analizar los comentarios

In [ ]:
# Procesamos todos los comentarios
resultados = clasificador(comentarios)

# Creamos un DataFrame para visualizar mejor los resultados
df_resultados = pd.DataFrame({
    'Comentario': comentarios,
    'Sentimiento': [r['label'] for r in resultados],
    'Confianza': [round(r['score'], 3) for r in resultados]
})

df_resultados

,Comentario,Sentimiento,Confianza
0,La comida llegó rapidísimo y estaba caliente. ...,POS,0.998
1,Tardaron DOS HORAS para traer una pizza fría. ...,NEG,0.999
2,"Todo bien, llegó en tiempo y forma. Nada para ...",POS,0.997
3,"Me encanta este servicio, siempre cumplen. Los...",POS,0.999
4,Se equivocaron con mi pedido OTRA VEZ. Ya es l...,NEG,0.999
5,"Increíble, dos horas esperando que lo entregue...",NEG,0.990
6,Llegó todo correctamente.,NEU,0.996
7,"Me llegó una empanada mordida, sin comentarios...",NEG,0.999
8,La empanada era pequeña.,NEU,0.997
9,Me dijeron que no pida acá pero me encantó,POS,0.999


### Paso 4: Identificar comentarios que requieren atención urgente

In [ ]:
# Filtramos solo los comentarios negativos
comentarios_urgentes = df_resultados[df_resultados['Sentimiento'] == 'NEG']

print("COMENTARIOS QUE REQUIEREN ATENCIÓN URGENTE:")
print("="*60)
for idx, fila in comentarios_urgentes.iterrows():
    print(f"\n[{idx+1}] {fila['Comentario']}")
    print(f"    Confianza: {fila['Confianza']*100:.1f}%")

COMENTARIOS QUE REQUIEREN ATENCIÓN URGENTE:

[2] Tardaron DOS HORAS para traer una pizza fría. Pésimo, no pido más acá.
    Confianza: 99.9%

[5] Se equivocaron con mi pedido OTRA VEZ. Ya es la tercera vez que pasa, un desastre total.
    Confianza: 99.9%

[6] Increíble, dos horas esperando que lo entreguen y la pizza llega con el queso pegado en la caja.
    Confianza: 99.0%

[8] Me llegó una empanada mordida, sin comentarios....
    Confianza: 99.9%


### Actividad práctica

Ahora es tu turno. Realizá las siguientes tareas:

1. **Agregá 3 comentarios nuevos** a la lista (pueden ser inventados o reales)
2. **Ejecutá nuevamente** las celdas de análisis
3. **Respondé:**
   - ¿El modelo clasificó correctamente tus comentarios?
   - ¿Hubo algún comentario que esperabas que clasifique diferente?
   - ¿Qué pasa si escribís un comentario con ironía? (probá con algo como "Genial, 3 horas esperando, lo mejor!")

Escribí tus comentarios nuevos en la siguiente celda:

Respuestas:
Si, el modelo clasificó correctamente los comentarios agregados, incluyendo aquellos de caracter sarcásticos y neutrales.
Si, "Increíble, dos horas esperando que lo entreguen y la pizza llega con el queso pegado en la caja." y "Me dijeron que no pida acá pero me encantó".

In [ ]:
# TUS COMENTARIOS ACÁ
mis_comentarios = [
    "Me dijeron que no pida acá pero me encantó",
    "La empanada era pequeña",
    "Me dijeron que no pida acá, pero me encantó"
]

# Analizamos
mis_resultados = clasificador(mis_comentarios)

# Mostramos
df_mis_resultados = pd.DataFrame({
    'Comentario': mis_comentarios,
    'Sentimiento': [r['label'] for r in mis_resultados],
    'Confianza': [round(r['score'], 3) for r in mis_resultados]
})

df_mis_resultados

,Comentario,Sentimiento,Confianza
0,Me dijeron que no pida acá pero me encantó,POS,0.999
1,La empanada era pequeña,NEU,0.991
2,"Me dijeron que no pida acá, pero me encantó",POS,0.999


---

## Ejercicio 2: Extracción Automática de Información de CVs

### Contexto del problema

Trabajás en el área de Recursos Humanos de una consultora argentina. Recibís decenas de CVs por día y necesitás extraer rápidamente información clave: nombres de candidatos, empresas donde trabajaron, universidades, ciudades, y tecnologías que manejan.

### Aplicación real

Este tipo de sistemas se usan en:
- Plataformas de empleo (LinkedIn, Bumeran, Computrabajo)
- Sistemas de ATS (Applicant Tracking Systems)
- Automatización de procesos de RRHH

### ¿Qué vamos a hacer?

1. Cargar un modelo de Reconocimiento de Entidades Nombradas (NER)
2. Procesar un fragmento de CV
3. Extraer automáticamente: personas, organizaciones, ubicaciones
4. Organizar la información de manera estructurada

---

### Paso 1: Cargar el modelo de NER

In [ ]:
# Cargamos un modelo de NER específico para español
# Este modelo fue entrenado para reconocer personas, organizaciones y lugares
extractor_ner = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"  # Agrupa tokens de la misma entidad
)

print("Modelo NER cargado correctamente.")

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Modelo NER cargado correctamente.


### Paso 2: Definir un fragmento de CV

Este es un ejemplo típico de cómo un candidato describe su experiencia:

In [ ]:
cv_texto = """María Fernández es una desarrolladora full-stack con 5 años de experiencia.
Estudió Ingeniería en Sistemas en la Universidad de Buenos Aires (UBA) y se graduó en 2018.
Trabajó como desarrolladora backend en Mercado Libre durante 3 años, donde lideró la migración
de sistemas legacy a microservicios. Posteriormente se unió a Globant como tech lead,
coordinando equipos distribuidos entre Buenos Aires y Córdoba. Tiene experiencia en Python,
Java, Docker y Kubernetes. Actualmente reside en Palermo, Ciudad de Buenos Aires."""

print("CV a procesar:")
print(cv_texto)

CV a procesar:
María Fernández es una desarrolladora full-stack con 5 años de experiencia.
Estudió Ingeniería en Sistemas en la Universidad de Buenos Aires (UBA) y se graduó en 2018.
Trabajó como desarrolladora backend en Mercado Libre durante 3 años, donde lideró la migración
de sistemas legacy a microservicios. Posteriormente se unió a Globant como tech lead,
coordinando equipos distribuidos entre Buenos Aires y Córdoba. Tiene experiencia en Python,
Java, Docker y Kubernetes. Actualmente reside en Palermo, Ciudad de Buenos Aires.


### Paso 3: Extraer entidades del CV

In [ ]:
# Procesamos el texto del CV
entidades = extractor_ner(cv_texto)

# Creamos un DataFrame para visualizar mejor
df_entidades = pd.DataFrame(entidades)

# Mostramos solo las columnas relevantes
df_entidades[['entity_group', 'word', 'score']].round(3)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,word,score
0,PER,María Fernández,1.000
1,MISC,Ingeniería en Sistemas,0.998
2,ORG,Universidad de Buenos Aires,0.998
3,ORG,UBA,0.983
4,MISC,Mercado Libre,0.789
5,ORG,Glo,1.000
6,ORG,##bant,0.821
7,LOC,Buenos Aires,1.000
8,LOC,Córdoba,1.000
9,MISC,Py,0.999


### Paso 4: Organizar la información por categoría

Ahora vamos a separar las entidades por tipo para que sea más fácil revisarlas:

In [ ]:
# Diccionario para organizar por tipo de entidad
info_extraida = {
    'PER': [],  # Personas
    'ORG': [],  # Organizaciones
    'LOC': []   # Ubicaciones
}

# Clasificamos cada entidad
for entidad in entidades:
    tipo = entidad['entity_group']
    palabra = entidad['word']
    if tipo in info_extraida:
        info_extraida[tipo].append(palabra)

# Mostramos la información organizada
print("INFORMACIÓN EXTRAÍDA DEL CV")
print("="*60)
print(f"\nCANDIDATO/A: {', '.join(info_extraida['PER'])}")
print(f"\nEMPRESAS/INSTITUCIONES: {', '.join(info_extraida['ORG'])}")
print(f"\nUBICACIONES: {', '.join(info_extraida['LOC'])}")

INFORMACIÓN EXTRAÍDA DEL CV

CANDIDATO/A: María Fernández

EMPRESAS/INSTITUCIONES: Universidad de Buenos Aires, UBA, Glo, ##bant

UBICACIONES: Buenos Aires, Córdoba, Palermo, Ciudad de Buenos Aires


### Actividad práctica

Ahora es tu turno. Realizá las siguientes tareas:

1. **Escribí un fragmento de CV ficticio** (o usá el tuyo si querés) con:
   - Nombre de la persona
   - Al menos 2 empresas o instituciones educativas
   - Al menos 2 ubicaciones (ciudades, barrios, países)

2. **Ejecutá el análisis** y verificá qué entidades detectó el modelo

3. **Respondé:**
   - ¿El modelo identificó correctamente todas las entidades?
   - ¿Hubo alguna entidad que no detectó? ¿Por qué creés que pasó?
   - ¿Detectó alguna entidad incorrectamente?

Escribí tu CV ficticio en la siguiente celda:

Por lo pronto, el modelo detectó algunas entidades correctamente, no obstante, algunas no han sido detectadas. Si ha detectado algunasi indentidades

In [ ]:
# TU CV FICTICIO ACÁ
mi_cv = """Currículum Vitae

Nombre: Lucía Fernández Álvarez
Nacionalidad: Argentina
Correo electrónico: lucia.fernandez.a@gmail.com

Teléfono: +54 11 4567-8920

Perfil Profesional

Analista de datos con experiencia en procesamiento de información, visualización y optimización de procesos internos. Apasionada por la tecnología, el aprendizaje continuo y la toma de decisiones basada en evidencia.

Formación Académica

Licenciatura en Ciencias de la Computación
Universidad de Buenos Aires (UBA) – Buenos Aires, Argentina
2016 – 2021

Curso de Especialización en Análisis de Datos
Centro de Innovación Tecnológica de Barcelona – Barcelona, España
2022 – 2023

Experiencia Laboral

Analista de Datos Junior
Empresa: Grupo TecnoSur – Palermo, Ciudad Autónoma de Buenos Aires, Argentina
Enero 2021 – Diciembre 2022

Implementación de reportes automatizados en Power BI.

Limpieza y procesamiento de grandes volúmenes de datos con Python.

Colaboración con el equipo de marketing para optimizar campañas digitales.

Data Analyst Senior
Empresa: DataVision Consulting – Madrid, España
Enero 2023 – Actualidad

Desarrollo de dashboards interactivos para clientes corporativos.

Análisis predictivo mediante modelos de machine learning.

Coordinación de un equipo de tres analistas junior.

Habilidades

Python (Pandas, NumPy, Matplotlib)

SQL

Power BI y Tableau

Inglés avanzado

Comunicación y trabajo en equipo"""

# Procesamos
mis_entidades = extractor_ner(mi_cv)

# Mostramos
df_mis_entidades = pd.DataFrame(mis_entidades)
print("Entidades detectadas:")
print(df_mis_entidades[['entity_group', 'word', 'score']].round(3))

# Organizamos
mi_info = {'PER': [], 'ORG': [], 'LOC': []}
for e in mis_entidades:
    if e['entity_group'] in mi_info:
        mi_info[e['entity_group']].append(e['word'])

print("\n" + "="*60)
print(f"CANDIDATO/A: {', '.join(mi_info['PER'])}")
print(f"EMPRESAS/INSTITUCIONES: {', '.join(mi_info['ORG'])}")
print(f"UBICACIONES: {', '.join(mi_info['LOC'])}")

Entidades detectadas:
   entity_group                                           word  score
0          MISC                                  ##culum Vitae  0.873
1           PER                        Lucía Fernández Álvarez  0.999
2           LOC                                      Argentina  0.982
3           PER                                             lu  0.953
4           PER                                          ##cia  0.718
5           PER                                      fernandez  0.722
6          MISC                                              +  0.706
7          MISC                                    Profesional  0.816
8          MISC                                Formación Acadé  0.911
9          MISC     Licenciatura en Ciencias de la Computación  0.992
10          ORG                    Universidad de Buenos Aires  0.997
11          ORG                                              U  0.997
12         MISC                                           ##BA  0.41

In [ ]:
# TU CV FICTICIO ACÁ
mi_cv = """
Nombre: VANESA CABRERA
PROFESSIONAL PROFILE
Bachelor's Degree in Data Science - April 2024 - currently in 2nd year. Programming in Python, SQL, Statistics, BI.
UNIR - Master's Degree in Music Therapy - March 2022 to December 2024
UAI - Bachelor's Degree in Psychology - March 2017 to March 2022
APADEA - Psychoeducational Intervention for students with ASD - March 2021 to December 2021 UAI - Bachelor's Degree in Education Sciences - March 2015 to February 2023
UAI - Teaching Degree in Education Sciences - March 2015 to December 2018
University of Morón - Appraiser and Public Auctioneer - March 2011 to February 2014 ISEA - Rural Administration Technician - March 1997 - December 1998
Instituto Lenguas Vivas - English Translation Studies - March 1990 - December 1992

LANGUAGES
English: C1 Level (Advanced)
French: B2 Level (Upper Intermediate)
Italian: Intermediate Level
EXPERIENCIA LABORAL
WORK EXPERIENCE
PEDAGOGICAL ADVISOR
Media School No. 3 Bernardino Rivadavia - March 2024 - currently active
Media School No. 28 Abraham Lincoln - February 2021 - March 2024
Detection of training needs of the teaching staff.
LEADER OF HUMAN RESOURCES, ORGANIZATIONAL AND CULTURAL PROJECTS
BILINGUAL EXECUTIVE HR ADVISOR - AMERICAS REGION
IATA, Miami


"""

# Procesamos
mis_entidades = extractor_ner(mi_cv)

# Mostramos
df_mis_entidades = pd.DataFrame(mis_entidades)
print("Entidades detectadas:")
print(df_mis_entidades[['entity_group', 'word', 'score']].round(3))

# Organizamos
mi_info = {'PER': [], 'ORG': [], 'LOC': []}
for e in mis_entidades:
    if e['entity_group'] in mi_info:
        mi_info[e['entity_group']].append(e['word'])

print("\n" + "="*60)
print(f"CANDIDATO/A: {', '.join(mi_info['PER'])}")
print(f"EMPRESAS/INSTITUCIONES: {', '.join(mi_info['ORG'])}")
print(f"UBICACIONES: {', '.join(mi_info['LOC'])}")

Entidades detectadas:
   entity_group                                       word  score
0           PER                                          V  0.962
1           PER                                     ##ANES  0.672
2          MISC        Bachelor ' s Degree in Data Science  0.884
3          MISC                                      Pytho  0.827
4          MISC                                        SQL  0.901
5          MISC                                     Statis  0.857
6          MISC                                         BI  0.903
7           ORG                                        UNI  0.588
8          MISC         Master ' s Degree in Music Therapy  0.990
9           LOC                                         UA  0.685
10         MISC                                        ##I  0.502
11         MISC          Bachelor ' s Degree in Psychology  0.980
12         MISC                                         AP  0.925
13         MISC                                     ##

---

## Ejercicio 3: Chatbot de Soporte Técnico Automático

### Contexto del problema

Trabajás en una empresa que vende electrodomésticos online. Los clientes suelen hacer preguntas frecuentes sobre garantías, envíos y devoluciones. Querés automatizar las respuestas a estas consultas usando un sistema de Question Answering (QA) que pueda responder basándose en la información de tus políticas.

### Aplicación real

Este tipo de sistemas se usan en:
- Chatbots de atención al cliente (WhatsApp, web)
- Sistemas de FAQ automáticas
- Asistentes virtuales corporativos

### ¿Qué vamos a hacer?

1. Cargar un modelo de Question Answering en español
2. Definir un contexto (políticas de la empresa)
3. Hacer preguntas sobre ese contexto
4. Generar respuestas automáticas

---

### Paso 1: Cargar los modelos necesarios

In [ ]:
from transformers import pipeline

# Modelo multilingüe que funciona perfectamente con español
qa_modelo = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2"
)

print("Modelo de QA cargado correctamente.")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


Modelo de QA cargado correctamente.


### Paso 2: Definir el contexto (políticas de la empresa)

Este es el documento que el chatbot va a usar para responder preguntas:

In [ ]:
politicas_empresa = """POLÍTICAS DE GARANTÍA Y DEVOLUCIONES - ELECTROHOGAR S.A.

GARANTÍA: Todos nuestros productos tienen garantía oficial de 12 meses contra defectos
de fábrica. La garantía comienza a contar desde la fecha de recepción del producto.
Para hacer válida la garantía, el cliente debe presentar la factura de compra original
y el producto no debe tener daños físicos causados por mal uso.

ENVÍOS: Realizamos envíos a todo el país. En Capital Federal y Gran Buenos Aires,
el envío demora entre 24 y 48 horas hábiles. Para el interior del país, el tiempo
de entrega es de 3 a 7 días hábiles. El envío es gratuito para compras superiores
a $50.000. Para montos menores, se cobra un adicional de $2.500.

DEVOLUCIONES: El cliente tiene 10 días corridos desde la recepción del producto para
solicitar una devolución si el artículo no cumple con sus expectativas. El producto
debe estar sin usar, en su empaque original y con todos sus accesorios. El cliente
debe hacerse cargo del costo de envío de devolución. Una vez recibido y verificado
el producto, reintegramos el 100% del dinero en un plazo de 15 días hábiles.
"""

print("Contexto cargado:")
print(politicas_empresa)

Contexto cargado:
POLÍTICAS DE GARANTÍA Y DEVOLUCIONES - ELECTROHOGAR S.A.

GARANTÍA: Todos nuestros productos tienen garantía oficial de 12 meses contra defectos
de fábrica. La garantía comienza a contar desde la fecha de recepción del producto.
Para hacer válida la garantía, el cliente debe presentar la factura de compra original
y el producto no debe tener daños físicos causados por mal uso.

ENVÍOS: Realizamos envíos a todo el país. En Capital Federal y Gran Buenos Aires,
el envío demora entre 24 y 48 horas hábiles. Para el interior del país, el tiempo
de entrega es de 3 a 7 días hábiles. El envío es gratuito para compras superiores
a $50.000. Para montos menores, se cobra un adicional de $2.500.

DEVOLUCIONES: El cliente tiene 10 días corridos desde la recepción del producto para
solicitar una devolución si el artículo no cumple con sus expectativas. El producto
debe estar sin usar, en su empaque original y con todos sus accesorios. El cliente
debe hacerse cargo del costo de envío

### Paso 3: Hacer preguntas al sistema

In [ ]:
# Definimos preguntas típicas de clientes
preguntas = [
    "¿Cuánto dura la garantía?",
    "¿Cuánto tarda el envío a Capital Federal?",
    "¿Cuántos días tengo para devolver un producto?",
    "¿Qué necesito para hacer válida la garantía?",
    "¿El envío es gratis?"
]

# Procesamos cada pregunta
print("RESPUESTAS AUTOMÁTICAS DEL CHATBOT")
print("="*60)

for pregunta in preguntas:
    respuesta = qa_modelo(question=pregunta, context=politicas_empresa)
    print(f"\nPREGUNTA: {pregunta}")
    print(f"RESPUESTA: {respuesta['answer']}")
    print(f"Confianza: {respuesta['score']*100:.1f}%")
    print("-"*60)

RESPUESTAS AUTOMÁTICAS DEL CHATBOT

PREGUNTA: ¿Cuánto dura la garantía?
RESPUESTA: el cliente debe presentar la factura de compra original
Confianza: 1.2%
------------------------------------------------------------

PREGUNTA: ¿Cuánto tarda el envío a Capital Federal?
RESPUESTA: 

Confianza: 0.5%
------------------------------------------------------------

PREGUNTA: ¿Cuántos días tengo para devolver un producto?
RESPUESTA: 10
Confianza: 65.9%
------------------------------------------------------------

PREGUNTA: ¿Qué necesito para hacer válida la garantía?
RESPUESTA: el cliente debe presentar la factura de compra original
Confianza: 24.3%
------------------------------------------------------------

PREGUNTA: ¿El envío es gratis?
RESPUESTA: $50.000
Confianza: 26.5%
------------------------------------------------------------


### Paso 4: Crear una función interactiva de chatbot

Vamos a crear una función que simule un chatbot completo:

In [ ]:
def chatbot_soporte(pregunta, contexto=politicas_empresa):
    """
    Función que simula un chatbot de soporte técnico.

    Args:
        pregunta (str): La pregunta del cliente
        contexto (str): El documento con las políticas de la empresa

    Returns:
        str: Respuesta formateada para el cliente
    """
    resultado = qa_modelo(question=pregunta, context=contexto)

    # Formateamos la respuesta de manera amigable
    confianza = resultado['score']

    if confianza > 0.5:
        respuesta = f"""Hola! Te respondo tu consulta:

{resultado['answer']}

¿Te fue útil esta información? Si necesitás más detalles, no dudes en consultarnos."""
    else:
        respuesta = """Hola! No encontré una respuesta clara a tu consulta en nuestras
políticas. Te recomiendo que te comuniques con nuestro equipo de atención al cliente
al 0800-XXX-XXXX para que puedan ayudarte mejor."""

    return respuesta

# Probamos la función
print(chatbot_soporte("¿Puedo devolver un producto después de 2 semanas?"))

Hola! No encontré una respuesta clara a tu consulta en nuestras
políticas. Te recomiendo que te comuniques con nuestro equipo de atención al cliente
al 0800-XXX-XXXX para que puedan ayudarte mejor.


### Actividad práctica

Ahora es tu turno. Realizá las siguientes tareas:

1. **Escribí 3 preguntas nuevas** que un cliente podría hacer sobre las políticas
2. **Probá el chatbot** con esas preguntas
3. **Modificá el contexto** (politicas_empresa) agregando información nueva, por ejemplo:
   - Formas de pago aceptadas
   - Horarios de atención
   - Información sobre instalación de productos
4. **Respondé:**
   - ¿Las respuestas fueron precisas?
   - ¿Hubo alguna pregunta que el modelo no pudo responder bien?
   - ¿Qué pasa si hacés una pregunta sobre algo que NO está en el contexto?

Usá las siguientes celdas para experimentar:

In [ ]:
politicas_empresa = """POLÍTICAS DE GARANTÍA Y DEVOLUCIONES - ELECTROHOGAR S.A.

GARANTÍA: Todos nuestros productos tienen garantía oficial de 24 meses contra defectos
de fábrica. No cubre roturas y mal funcionamiento por falta de cuidado como ser roturas por caídas.
La garantía comienza a contar desde la fecha de recepción del producto.
Para hacer válida la garantía, el cliente debe presentar la factura de compra original
y el producto no debe tener daños físicos causados por mal uso.

ENVÍOS: Realizamos envíos a todo el país. En Capital Federal y Gran Buenos Aires,
el producto se entrega dentro de las 72 horas hábiles de 8 a 12. en caso de no estar
 presente en el domicilio para recibir el producto el dia de la entrega, se deberá abonar el envío.
 Para el interior del país, el tiempo
de entrega es de 3 a 7 días hábiles. El envío es gratuito para compras superiores
a $50.000. Para montos menores, se cobra un adicional de $2.500.

DEVOLUCIONES: El cliente tiene 10 días corridos desde la recepción del producto para
solicitar una devolución si el artículo no cumple con sus expectativas. El producto
debe estar sin usar, en su empaque original y con todos sus accesorios. El cliente
debe hacerse cargo del costo de envío de devolución. Una vez recibido y verificado
el producto, reintegramos el 100% del dinero en un plazo de 15 días hábiles.
"""

print("Contexto cargado:")
print(politicas_empresa)

Contexto cargado:
POLÍTICAS DE GARANTÍA Y DEVOLUCIONES - ELECTROHOGAR S.A.

GARANTÍA: Todos nuestros productos tienen garantía oficial de 24 meses contra defectos
de fábrica. No cubre roturas y mal funcionamiento por falta de cuidado como ser roturas por caídas.
La garantía comienza a contar desde la fecha de recepción del producto.
Para hacer válida la garantía, el cliente debe presentar la factura de compra original
y el producto no debe tener daños físicos causados por mal uso.

ENVÍOS: Realizamos envíos a todo el país. En Capital Federal y Gran Buenos Aires,
el producto se entrega dentro de las 72 horas hábiles de 8 a 12. en caso de no estar
 presente en el domicilio para recibir el producto el dia de la entrega, se deberá abonar el envío.
 Para el interior del país, el tiempo
de entrega es de 3 a 7 días hábiles. El envío es gratuito para compras superiores
a $50.000. Para montos menores, se cobra un adicional de $2.500.

DEVOLUCIONES: El cliente tiene 10 días corridos desde la rec

In [ ]:
# Cargar modelo
qa_modelo = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2"
)

politicas_empresa = """
POLITICAS DE LA EMPRESA

1. POLITICA DE DEVOLUCIONES
Todos nuestros productos tienen garantía oficial de 12 meses contra defectos
de fábrica. La garantía comienza a contar desde la fecha de recepción del producto.
- Formas de pago: se puede pagar en efectivo, con tarjeta de crédito en cuotas y a través de transferencia
- Horarios de atención: el horario de atención al cliente es de 9 a 18 horas.
- Horario de entrega: los pedidos se entregan de 9 a 12 horas.
Los clientes tienen derecho a devolver productos dentro de los 30 dias corridos desde la fecha de compra.
Las devoluciones despues de 2 semanas son aceptadas si el producto tiene defectos.
El reembolso se procesa en 10 dias habiles.

2. POLITICA DE ENVIOS
Envio Estandar: 5 a 7 dias, costo 500 pesos.
Envio Express: 2 a 3 dias, costo 1200 pesos.
Envio gratis en compras mayores a 5000 pesos.

3. GARANTIA
Todos los productos tienen garantia de 12 meses contra defectos de fabricacion.
"""

def chatbot_diagnostico(pregunta, contexto=politicas_empresa):
    print(f"\nPROCESANDO: {pregunta}")

    try:
        resultado = qa_modelo(question=pregunta, context=contexto)

        print(f"Respuesta encontrada: {resultado['answer']}")
        print(f"Confianza: {resultado['score']*100:.2f}%")
        print(f"Posicion en texto: {resultado['start']} - {resultado['end']}")

        # Mostrar contexto alrededor de la respuesta
        inicio = max(0, resultado['start'] - 50)
        fin = min(len(contexto), resultado['end'] + 50)
        print(f"Contexto: ...{contexto[inicio:fin]}...")

        return resultado

    except Exception as e:
        print(f"ERROR: {e}")
        return None

# PROBAR CON DIAGNOSTICO
preguntas = [
    "Cuantos dias tengo para devolver?",
    "combien ca coute le service de delivery?"
    "how much does the standard delivery cost?",
    "Cuanto dura la garantia?",
    "cuales son las formas de pago?",
    "Cuando se entrega el pedido una vez realizado el pago?",
    "cual es el horario de atención al cliente?",
    "cual es el horario de entrega?",
    "Se me cayó al piso el producto, me cubre la garantía?",
    "DE cuanto es la garantía de los productos?",
    "hacen envíos a todo el país?",
    "Qué ocurre si no estoy presente para recibir el pedido el día de la entrega?"
]

for pregunta in preguntas:
    resultado = chatbot_diagnostico(pregunta)
    print("="*60)

Device set to use cpu



PROCESANDO: Cuantos dias tengo para devolver?
Respuesta encontrada: 30
Confianza: 38.38%
Posicion en texto: 526 - 528
Contexto: ...tienen derecho a devolver productos dentro de los 30 dias corridos desde la fecha de compra.
Las devol...

PROCESANDO: combien ca coute le service de delivery?how much does the standard delivery cost?
Respuesta encontrada: 1200 pesos
Confianza: 0.52%
Posicion en texto: 797 - 807
Contexto: ...costo 500 pesos.
Envio Express: 2 a 3 dias, costo 1200 pesos.
Envio gratis en compras mayores a 5000 pesos.

3...

PROCESANDO: Cuanto dura la garantia?
Respuesta encontrada: desde la fecha de recepción del producto
Confianza: 15.30%
Posicion en texto: 173 - 213
Contexto: ...efectos
de fábrica. La garantía comienza a contar desde la fecha de recepción del producto.
- Formas de pago: se puede pagar en efectivo, co...

PROCESANDO: cuales son las formas de pago?
Respuesta encontrada: Horarios de atención
Confianza: 0.61%
Posicion en texto: 324 - 344
Contexto: ...e crédito 

---

## Ejercicio 4: Desafío Autónomo - Análisis de Reseñas de Restaurantes

### Contexto del problema

Sos el encargado de marketing digital de una cadena de restaurantes porteña. Querés implementar un sistema inteligente que procese automáticamente las reseñas que los clientes dejan en Google Maps y redes sociales para:

1. Identificar si la reseña es positiva, negativa o neutral
2. Extraer información clave: nombres de platos mencionados, ubicaciones de las sucursales, nombres de empleados destacados
3. Responder automáticamente a preguntas frecuentes basándose en el menú y políticas del restaurante

### Aplicación real

Este tipo de sistemas combinados se usan en:
- Gestión de reputación online para cadenas de restaurantes
- Análisis de feedback de clientes en hotelería y turismo
- Sistemas de CRM (Customer Relationship Management) inteligentes
- Plataformas de delivery con análisis de satisfacción del cliente

---

### Tu tarea

**Este ejercicio lo tenés que resolver completamente solo**, aplicando todo lo que aprendiste en los ejercicios anteriores. No hay código de ejemplo, solo las instrucciones.

### Parte 1: Análisis de sentimiento de reseñas (30%)

1. Creá una lista con al menos 5 reseñas ficticias de clientes sobre un restaurante argentino (podés inventarlas o buscar reales)
2. Cargá un modelo de análisis de sentimientos en español (buscá en Hugging Face)
3. Clasificá cada reseña y mostrá los resultados en un DataFrame
4. Identificá cuántas reseñas son positivas, negativas y neutrales (si el modelo lo soporta)

**Pistas:**
- Usá `pipeline("text-classification", model=...)`
- Recordá importar `pandas` para crear el DataFrame
- Modelos sugeridos: `finiteautomata/beto-sentiment-analysis` o `pysentimiento/robertuito-sentiment-analysis`

---

### Parte 2: Extracción de información (40%)

1. Tomá 2 de las reseñas que creaste (las más largas y detalladas)
2. Cargá un modelo de NER en español
3. Extraé todas las entidades nombradas de esas reseñas
4. Organizá la información en categorías (personas, lugares, organizaciones)
5. Bonus: ¿Se mencionan nombres de platos? (Nota: el modelo podría no detectarlos como entidades, reflexioná sobre por qué)

**Pistas:**
- Usá `pipeline("ner", model=..., aggregation_strategy="simple")`
- Modelo sugerido: `mrm8488/bert-spanish-cased-finetuned-ner`
- Recordá iterar sobre los resultados para organizarlos por tipo

---

### Parte 3: Sistema de preguntas y respuestas (30%)

1. Escribí un texto con información del restaurante (menú, horarios, ubicación, políticas de reservas, precios promedio, etc.). Mínimo 4-5 oraciones.
2. Cargá un modelo de Question Answering en español
3. Formulá al menos 4 preguntas que un cliente podría hacer
4. Generá respuestas automáticas usando el modelo
5. Mostrá cada pregunta con su respuesta y el nivel de confianza del modelo

**Pistas:**
- Usá `pipeline("question-answering", model=...)`
- Modelo sugerido: `PlanTL-GOB-ES/roberta-base-bne-sqac`
- La función necesita dos parámetros: `question=` y `context=`

---

### Bonus (opcional): Integración completa

Si terminaste las tres partes, intentá crear una función que:
1. Reciba una reseña de cliente como input
2. Analice el sentimiento
3. Extraiga entidades mencionadas
4. Genere un resumen estructurado

Por ejemplo:
```
RESEÑA: "Fui ayer a la sucursal de Palermo y el mozo Juan me atendió bárbaro..."

ANÁLISIS:
- Sentimiento: POSITIVO (95% confianza)
- Empleado mencionado: Juan
- Sucursal: Palermo
- Recomendación: Enviar agradecimiento personalizado
```

---

### Criterios de evaluación

Evaluá tu propio trabajo considerando:

1. **Funcionalidad (50%):** ¿El código funciona sin errores? ¿Completaste las tres partes?
2. **Calidad de datos (20%):** ¿Las reseñas y preguntas son realistas? ¿El contexto tiene información útil?
3. **Presentación (20%):** ¿Los resultados se muestran de forma clara? ¿Usaste DataFrames o print statements organizados?
4. **Reflexión crítica (10%):** ¿Analizaste la calidad de las predicciones? ¿Identificaste limitaciones?

---

### Espacio para tu solución

Usá las celdas siguientes para resolver el desafío. Podés crear todas las celdas que necesites.


### Parte 1 de ejercicio 4

In [ ]:
# Parte 1: Análisis de sentimiento de reseñas (30%)

resenas = ["El asado estaba exquisito, la mejor carne que probé en años. El ambiente muy acogedor y Hugo me atendio muuuuuy  bien.",
    "Pésima atención, esperamos más de una hora por la comida. No vuelvo más.",
    "Las empanadas estaban ricas pero nada del otro mundo. Precio normal.",
    "¡Increíble experiencia! El choripán casero es espectacular. Muy recomendable. Imperdible este restaurante ubicado en el centro de Palermo, Buenos Aires",
    "La milanesa estaba quemada y el mozo fue muy antipático. Decepcionante.",
    "Buen lugar, precios accesibles. La pizza de fugazzeta cumple.",
    "Horrible, la carne estaba dura y fría. Nunca más voy a ese lugar.",
    "Me encantó todo, desde el fernet hasta los postres. Volveré seguro."]

model = pipeline(
    "text-classification",
    model="finiteautomata/beto-sentiment-analysis"
)

# Procesamos todos los comentarios
resultados = model(resenas)
print(resultados)

labels = []
scores = []

for i in resultados:
  labels.append(i["label"])
  scores.append(i["score"])


# Creamos un DataFrame para visualizar mejor los resultados
df_resultados = pd.DataFrame({
    'Comentario': resenas,
    'Sentimiento': labels,
    'Confianza': scores
})

df_resultados

Device set to use cpu


[{'label': 'POS', 'score': 0.9987390637397766}, {'label': 'NEG', 'score': 0.9993104934692383}, {'label': 'POS', 'score': 0.990210771560669}, {'label': 'POS', 'score': 0.998494029045105}, {'label': 'NEG', 'score': 0.9826734662055969}, {'label': 'POS', 'score': 0.9988034963607788}, {'label': 'NEG', 'score': 0.9992824196815491}, {'label': 'POS', 'score': 0.9988529682159424}]


,Comentario,Sentimiento,Confianza
0,"El asado estaba exquisito, la mejor carne que probé en años. El ambiente muy...",POS,0.998739
1,"Pésima atención, esperamos más de una hora por la comida. No vuelvo más.",NEG,0.999310
2,Las empanadas estaban ricas pero nada del otro mundo. Precio normal.,POS,0.990211
3,¡Increíble experiencia! El choripán casero es espectacular. Muy recomendable...,POS,0.998494
4,La milanesa estaba quemada y el mozo fue muy antipático. Decepcionante.,NEG,0.982673
5,"Buen lugar, precios accesibles. La pizza de fugazzeta cumple.",POS,0.998803
6,"Horrible, la carne estaba dura y fría. Nunca más voy a ese lugar.",NEG,0.999282
7,"Me encantó todo, desde el fernet hasta los postres. Volveré seguro.",POS,0.998853


### Parte 2 de ejercicio 4

In [ ]:
# Parte 2: Extracción de información (40%)

resenas_ordenadas = sorted(resenas, key=len, reverse=True)
resenas_ext = []
for i in range(2):
  resenas_ext.append(resenas_ordenadas[i])

extractor_ner = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"  # Agrupa tokens de la misma entidad
)

entidades = extractor_ner(resenas_ext[0])
entidades2 = extractor_ner(resenas_ext[1])

df_entidades = pd.DataFrame(entidades)
df_entidades2 = pd.DataFrame(entidades2)

df_entidades_final = pd.concat([df_entidades, df_entidades2])

# Mostramos solo las columnas relevantes
df_entidades_final[['entity_group', 'word', 'score']].round(3)


Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,word,score
0,LOC,Palermo,1.000
1,LOC,Buenos Aires,0.999
0,PER,Hugo,1.000


### Parte 3 de ejercicio 4

In [ ]:
# PARTE 3: QUESTION ANSWERING

from transformers import pipeline

texto =  """La Trattoria del Sol es un restaurante italiano ubicado en Avenida Santa Fe 2450, barrio de Recoleta, Buenos Aires.
    Nuestro horario de atención es de martes a domingo de 12:00 a 16:00 para almuerzos y de 20:00 a 00:00 para cenas.
    Los lunes permanecemos cerrados. El precio promedio por persona es de $15000 a $25000 pesos argentinos, dependiendo del menú elegido.
    Nuestro menú incluye platos tradicionales italianos como: pasta fresca casera (ravioles, ñoquis, fetuccini), risottos cremosos,
    pizzas a la piedra, ossobuco a la milanesa, y tiramisú artesanal de postre. También ofrecemos opciones vegetarianas y veganas.
    Se reserva a través de whatsapp al número 011-4805-3322 o a través de nuestra página web con al menos 24 horas de anticipación.
    Los fines de semana recomendamos reservar con 48 horas de anticipación debido a la alta demanda. Aceptamos grupos de hasta 12 personas.
    Contamos con estacionamiento privado para clientes y aceptamos todas las tarjetas de crédito y débito. También ofrecemos servicio de delivery
    en un radio de 5 kilómetros. El ambiente es acogedor con decoración rústica italiana y música en vivo los viernes y sábados por la noche."""

# Cargar modelo de Question Answering en español
print("Cargando modelo de Question Answering...")
# Reverting to a model that showed signs of loading previously
qa_modelo = pipeline("question-answering", model="deepset/roberta-base-squad2")
print("Modelo de QA cargado!")


Preguntas = [
    "¿Cuál es el horario de atención del restaurante?",
    "¿Dónde está ubicado La Trattoria del Sol?",
    "¿Cuánto cuesta comer en el restaurante?",
    "until what time is it open for lunch?",
    "¿Qué platos tienen en el menú?",
    "¿Cómo puedo hacer una reserva?",
    "¿Tienen opciones vegetarianas?"
]

for question in Preguntas:
    result = qa_modelo(question=question, context=texto)
    print(f"Pregunta: {question}")
    print(f"Respuesta: {result['answer']}")
    print(f"Confianza: {result['score']:.4f}\n")

Cargando modelo de Question Answering...


Device set to use cpu


Modelo de QA cargado!
Pregunta: ¿Cuál es el horario de atención del restaurante?
Respuesta: 24 horas de anticipación
Confianza: 0.1750

Pregunta: ¿Dónde está ubicado La Trattoria del Sol?
Respuesta: Buenos Aires
Confianza: 0.0122

Pregunta: ¿Cuánto cuesta comer en el restaurante?
Respuesta: La Trattoria del Sol
Confianza: 0.0527

Pregunta: until what time is it open for lunch?
Respuesta: 12:00 a 16:00
Confianza: 0.0611

Pregunta: ¿Qué platos tienen en el menú?
Respuesta: tradicionales italianos
Confianza: 0.0644

Pregunta: ¿Cómo puedo hacer una reserva?
Respuesta: Los fines de semana
Confianza: 0.0072

Pregunta: ¿Tienen opciones vegetarianas?
Respuesta: También
Confianza: 0.0693



In [ ]:
# BONUS (OPCIONAL): INTEGRACIÓN COMPLETA

"""Si terminaste las tres partes, intentá crear una función que:
1. Reciba una reseña de cliente como input
2. Analice el sentimiento
3. Extraiga entidades mencionadas
4. Genere un resumen estructurado

Por ejemplo:
```
RESEÑA: "Fui ayer a la sucursal de Palermo y el mozo Juan me atendió bárbaro..."

ANÁLISIS:
- Sentimiento: POSITIVO (95% confianza)
- Empleado mencionado: Juan
- Sucursal: Palermo
- Recomendación: Enviar agradecimiento personalizado"""



### Reflexión personal

Después de completar el ejercicio, respondé estas preguntas:

1. **¿Qué fue lo más difícil del ejercicio?**
   - Integrar todos los pasos de manera de obtener un procedimiento completo

2. **¿Encontraste alguna limitación en los modelos?**
   - Si, falta de precisión en la clasificación de personas, localidades e identificación de comentarios positivos, negativos y neutros. En el caso de análisis de CV, no se lograba identificar el nombre del participante, y en cuanto a la longitud, tiene un límite de tokens a ser procesados.

3. **¿Cómo podrías mejorar este sistema para un caso real?**
   - En el caso de reseñas de un restaurante se pueden utilizar modelos que sean multilingües de manera de poder analizar reseñas en distintos idiomas, ya que los restaurantes suelen ser visitados por extranjeros de distitntos países al igual que los hoteles.

4. **¿Qué otras aplicaciones se te ocurren para estas técnicas en el contexto argentino?**
   - Dependiendo el ámbito de uso, se podría implementar en el sector público, para analizar rclamos del 147, monitorear redes sociales gubernamentales o en el ámbito privado, cualquier red social de organizaciones multinacionales.
   Puede ser de mucha utilidad para aquellas organizaciones que brindan serivcios o venden productos, para estar pendientes de la satisfacción del cliente en relación a los productos adquiridos, y abordar las reseñas negativas de caracter urgente para apaliar el descontento del consumidor. Asimismo en medicina, para clasificar pacientes en función de la urgencia de la patologia o enfermedad y así priorizar la atención médica. Analizar reseñas de hoteles, restaurantes, servicios ofrecidos, etc.
---

---

## Reflexiones Finales del Curso

Completaste los cuatro ejercicios de NLP con Transformers, incluyendo un desafío autónomo. Vamos a reflexionar sobre lo aprendido:

### Ejercicio 1 - Moderación de Comentarios
- **Aprendiste:** A clasificar texto usando modelos de análisis de sentimientos
- **Aplicación:** Moderación automática de redes sociales, priorización de tickets de soporte
- **Limitaciones:** Los modelos pueden tener problemas con ironía, sarcasmo o lenguaje muy coloquial

### Ejercicio 2 - Extracción de CVs
- **Aprendiste:** A extraer entidades nombradas (personas, organizaciones, lugares) de texto
- **Aplicación:** Automatización de RRHH, análisis de documentos, extracción de información estructurada
- **Limitaciones:** Algunos modelos pueden confundir entidades o no detectar nombres poco comunes

### Ejercicio 3 - Chatbot de Soporte
- **Aprendiste:** A usar Question Answering para responder preguntas basadas en un contexto
- **Aplicación:** Chatbots, asistentes virtuales, sistemas de FAQ automáticas
- **Limitaciones:** El modelo solo puede responder sobre información presente en el contexto

### Ejercicio 4 - Desafío Autónomo
- **Aprendiste:** A combinar múltiples técnicas de NLP para resolver un problema real completo
- **Aplicación:** Sistemas integrados de análisis de feedback, gestión de reputación online
- **Habilidad clave:** Autonomía para investigar, implementar y evaluar soluciones de NLP

### Próximos pasos sugeridos

1. **Explorá más modelos** en [Hugging Face Hub](https://huggingface.co/models)
2. **Combiná técnicas:** Por ejemplo, usá clasificación de sentimientos + QA para un chatbot más inteligente
3. **Experimentá con otros idiomas** o dialectos regionales
4. **Investigá fine-tuning:** Aprendé a ajustar modelos con tus propios datos
5. **Desarrollá un proyecto propio:** Elegí un problema real que te interese y aplicá estas técnicas

---

## Glosario Técnico

### Conceptos fundamentales

**Transformer**  
Arquitectura de red neuronal basada en mecanismos de atención que revolucionó el NLP en 2017. Permite procesar secuencias de texto completas simultáneamente en lugar de palabra por palabra.

**Pipeline**  
Interfaz de alto nivel en Hugging Face que encapsula todo el proceso de preprocesamiento, inferencia y postprocesamiento de un modelo. Facilita el uso de modelos preentrenados con pocas líneas de código.

**Modelo preentrenado**  
Modelo de machine learning que fue entrenado previamente con grandes cantidades de datos. Puede usarse directamente o ajustarse (fine-tuning) para tareas específicas.

**Tokenización**  
Proceso de dividir texto en unidades más pequeñas (tokens) que el modelo puede procesar. Puede ser a nivel de palabras, subpalabras o caracteres.

### Tareas de NLP

**Text Classification (Clasificación de texto)**  
Tarea de asignar una o más etiquetas a un texto. Incluye análisis de sentimientos, detección de spam, clasificación de temas, etc.

**Sentiment Analysis (Análisis de sentimientos)**  
Subtipo de clasificación que identifica la polaridad emocional de un texto (positivo, negativo, neutral). Se usa en redes sociales, reseñas de productos, atención al cliente.

**Named Entity Recognition - NER (Reconocimiento de entidades nombradas)**  
Tarea de identificar y clasificar nombres propios en texto: personas (PER), organizaciones (ORG), ubicaciones (LOC), fechas, cantidades monetarias, etc.

**Question Answering - QA (Respuesta a preguntas)**  
Tarea de responder preguntas en lenguaje natural basándose en un contexto dado. El modelo extrae la respuesta directamente del texto proporcionado.

**Text Generation (Generación de texto)**  
Tarea de crear texto nuevo de manera coherente a partir de un prompt inicial. Incluye completado de texto, escritura creativa, chatbots conversacionales.

**Summarization (Resumen automático)**  
Tarea de condensar un texto largo en una versión más corta manteniendo la información más importante.

**Translation (Traducción automática)**  
Tarea de traducir texto de un idioma a otro usando modelos de secuencia a secuencia.

### Componentes técnicos

**Embedding (Representación vectorial)**  
Representación numérica de palabras o tokens como vectores en un espacio multidimensional. Palabras con significados similares tienen embeddings cercanos.

**Attention (Atención)**  
Mecanismo que permite al modelo enfocarse en diferentes partes del input al procesar cada elemento. Es el componente clave de la arquitectura Transformer.

**Fine-tuning (Ajuste fino)**  
Proceso de tomar un modelo preentrenado y entrenarlo adicionalmente con datos específicos de tu dominio para mejorar su desempeño en tu tarea particular.

**Inference (Inferencia)**  
Proceso de usar un modelo ya entrenado para hacer predicciones sobre datos nuevos. No implica entrenamiento, solo aplicación del modelo.

**Score / Confidence (Puntuación / Confianza)**  
Valor numérico (generalmente entre 0 y 1) que indica qué tan seguro está el modelo de su predicción. Valores más altos indican mayor confianza.

### Modelos mencionados

**BETO**  
Versión de BERT (Bidirectional Encoder Representations from Transformers) entrenada específicamente con texto en español. Usado en análisis de sentimientos y otras tareas de clasificación.

**RoBERTa**  
Variante optimizada de BERT con mejoras en el proceso de preentrenamiento. Usado para múltiples tareas de NLP en español.

**BERT (Bidirectional Encoder Representations from Transformers)**  
Modelo Transformer que lee texto bidireccionalmente (izquierda a derecha y derecha a izquierda simultáneamente) para comprender mejor el contexto.

**GPT (Generative Pre-trained Transformer)**  
Familia de modelos diseñados específicamente para generación de texto. Leen texto de izquierda a derecha y predicen la siguiente palabra.

### Plataformas y librerías

**Hugging Face**  
Plataforma y empresa que desarrolla herramientas de NLP de código abierto. Su librería Transformers es el estándar de facto para trabajar con modelos de lenguaje.

**Hugging Face Hub**  
Repositorio online con miles de modelos preentrenados, datasets y demos interactivas. Permite compartir y descargar modelos fácilmente.

**PyTorch / TensorFlow**  
Frameworks de deep learning usados como backend por la librería Transformers. PyTorch es más común en investigación, TensorFlow en producción.

### Métricas y evaluación

**Label (Etiqueta)**  
Categoría asignada por el modelo a un texto. En análisis de sentimientos: POS (positivo), NEG (negativo), NEU (neutral).

**Aggregation strategy (Estrategia de agregación)**  
En NER, método para combinar tokens que pertenecen a la misma entidad. Por ejemplo, "Buenos" y "Aires" se agrupan en una sola entidad "Buenos Aires".

**Context (Contexto)**  
En QA, el documento o párrafo que contiene la información necesaria para responder la pregunta. El modelo busca la respuesta dentro de este contexto.

---

## Recursos adicionales

- **Documentación oficial de Transformers:** https://huggingface.co/docs/transformers
- **Modelos en español:** https://huggingface.co/models?language=es
- **Curso gratuito de Hugging Face:** https://huggingface.co/course
- **Comunidad en español:** https://huggingface.co/spaces

---

*Este cuaderno fue diseñado con fines educativos para estudiantes de NLP en Argentina.*